---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32

In [5]:
import pandas as pd
import numpy as np

def blight_model():
#      The target variable is compliance
#     loads dataframes
    train_df = pd.read_csv('train.csv', encoding='latin1')
        
    test_df = pd.read_csv('test.csv')
    
    addresses_df = pd.read_csv('addresses.csv')
    
    latlons_df = pd.read_csv('latlons.csv')
    
#     FEATURE 1 LATE FEE LABEL
#     .apply(lambda x: 1 if x >=  renewable_median else 0)
# creating, what I think is, a classifer of either true or false based on if there was a late fee or not
# if there was a late fee, that is, if the amount is greater than 0 for the late fee, it gets a 1, otherwise, it is a 0
    train_df['late_fee_class'] = train_df['late_fee'].apply(lambda x: 1 if x > 0 else 0)
    
    
    
#     reading through the forum, it seems that really 3-4 numerical features may be enough.

# feature selection is the KEY Part according to the forum discussion
# lots of people indicate that 3 - 4 features are all that are a needed to train the model 

# leaning towards violation_zip_code
# deriving late_fee into a label of TRUE or FALSE (0 or 1) if there is a late fee potentially
# zip_code of the violator
# fine_amount

# note from forum:
# I selected 4 features and ran a GridSearchCV to find the best parameters. 
# It took me hours to realize that I was displaying the first column of predict_proba instead of the second one which gave me an AUC of 0.2...

#  RandomForest Classifier with default setting. was a recommendation 

# preprocessing data notes
# dropping columns with lots of NULL /NaN are key...apparenlty there may be some that are straight up all NULL, so not using that feature is obvious

# then converting the null or NaN to something else

# ultimately running the model should be pretty straight forward I guess once everything is processed correclty


#      training set with no null complaince 
    train_df = train_df[train_df['compliance'].notna()]
    

    
    
#     return  train_df['late_fee_class']
    return  train_df[['late_fee_class', 'late_fee', 'compliance']]
blight_model()

late_fee_class  late_fee  compliance
0                    1      25.0         0.0
1                    1      75.0         1.0
5                    1      25.0         0.0
6                    1      75.0         0.0
7                    1      10.0         0.0
8                    1      10.0         0.0
9                    1      75.0         0.0
12                   1      75.0         0.0
13                   1      75.0         0.0
14                   1      75.0         0.0
18                   1      10.0         1.0
19                   1      10.0         1.0
21                   1      75.0         0.0
22                   1      75.0         0.0
23                   1      75.0         0.0
26                   0       0.0         0.0
28                   1      25.0         0.0
30                   1      25.0         0.0
31                   1      25.0         0.0
33                   1      25.0         0.0
34                   1      25.0         0.0
35                   1      25.0         0.0
37                   1      25.0         1.0
38                   1      25.0         0.0
39                   1      25.0         0.0
40                   1      25.0         0.0
41                   0       0.0         1.0
42                   0       0.0         0.0
43                   1      25.0         0.0
44                   0       0.0         1.0
...                ...       ...         ...
250244               1      75.0         0.0
250249               1      25.0         0.0
250250               0       0.0         0.0
250254               1      50.0         0.0
250255               1      25.0         0.0
250256               1      25.0         0.0
250257               1     100.0         0.0
250258               1     100.0         0.0
250259               1     100.0         0.0
250261               1      10.0         0.0
250262               1      10.0         0.0
250264               1       5.0         0.0
250265               0       0.0         1.0
250266               1       5.0         0.0
250267               1       5.0         0.0
250268               1       5.0         0.0
250269               1       5.0         0.0
250270               1       5.0         0.0
250271               1       5.0         0.0
250272               1       5.0         0.0
250273               1       5.0         0.0
250274               1     100.0         0.0
250275               1       5.0         0.0
250276               1      10.0         0.0
250277               1      50.0         0.0
250278               1      20.0         0.0
250287               1     100.0         0.0
250288               1      50.0         0.0
250290               0       0.0         1.0
250293               1      20.0         0.0

[159880 rows x 3 columns]

In [ ]:
34